In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input

In [18]:
##################
### LOAD IMAGE ###
##################

image_path = '../NEW_DATASET/test/snl_cast.jpg'
export_name = 'chris_masked_normal'
write_bool = False

network = cv2.dnn.readNetFromCaffe("../converging_models/MODELS/deploy.prototxt", "../converging_models/MODELS/res10_300x300_ssd_iter_140000_fp16.caffemodel")

############################
### LABEL MULTIPLE FACES ###
############################
dnn_label = ''
scale_factor = 1
dnn_boxColor = (255,0,0)
face_recognition_conf = 0.9

dnn_image = cv2.imread(image_path)
dnn_image = cv2.resize(dnn_image, (0,0), fx=scale_factor, fy=scale_factor)


(height, width) = dnn_image.shape[:2]
blob = cv2.dnn.blobFromImage(dnn_image, scalefactor=1.0, size=(width, height), mean=(104.0, 117.0, 123.0))
network.setInput(blob)
detections = network.forward()

for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > face_recognition_conf:
        box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
        left = int(box[0])
        top = int(box[1])
        right = int(box[2])
        bottom = int(box[3])

        cropped_image = dnn_image[top:bottom, left:right]

        cv2.rectangle(dnn_image, (left, top), (right, bottom), dnn_boxColor, 2)

if write_bool:
    cv2.imwrite('../NEW_DATASET/test/export/' + export_name + '.jpg', dnn_image)

cv2.imshow("DNN", dnn_image)
cv2.waitKey(1)

-1